In [1]:
import os, sys
from pathlib import Path
script_dir = Path(os.path.dirname(os.path.abspath('')))
module_dir = str(script_dir)
sys.path.insert(0, module_dir + '/modules')
print(module_dir)

# import the rest of the modules
%matplotlib nbagg
import numpy as np
import tensorflow as tf 
import matplotlib.pyplot as plt
import arch
import pandas as pd
import tensorflow_probability as tfp
import time  
import sim
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.animation as animation
from mpl_toolkits.axes_grid1 import make_axes_locatable

dim = 3
n_particles = int(1e6)
n_subdivisions = 30
n_int_subdivs = 10000
save_folder = '../data/L63'
n_steps = 10
n_repeats = 10
dt = 0.01
r = 1.0
alpha, beta, rho = 10., 8./3., 28.
sigma = 10.
D = sigma**2 / 2.
t = dt * n_steps

# low=[-20., -20., -20.]
# high=[20., 30., 80.]
# domain = [low, high]

def mu_np(X):
    x, y, z = np.split(X, dim, axis=-1)
    p = alpha * (y - x) 
    q = x * (rho - z) - y 
    r = x * y - beta * z
    return np.concatenate([p, q, r], axis=-1)


X0 =  tfp.distributions.MultivariateNormalDiag(scale_diag=r*tf.ones(dim)).sample(n_particles).numpy()
mc_prob = sim.MCProb(save_folder, n_subdivisions, mu_np, sigma, X0, tick_size=20, title_size=20, cbar_tick_size=15)
mc_prob.ready(n_steps=n_steps, dt=dt, lims=None)
#mc_prob.compute_all(n_steps=n_steps, dt=dt)

C:\Users\pinak\Documents\GitHub\non-grad3D
Time taken by propagate is 2.845925807952881 seconds
Time taken by set_grid is 2.5720772743225098 seconds
Time taken by assign_pts is 3.1127254962921143 seconds
Time taken by ready is 8.534601211547852 seconds


In [ ]:
def get_net(iter):
    net = arch.LSTMForgetNet(num_nodes=50, num_blocks=3)
    if iter < 500:
        net.load_weights('../data/L63-true-vs-learned/init/L63_{}'.format(iter)).expect_partial()
    elif iter < 10000:
        net.load_weights('../data/L63-true-vs-learned/L63_{}'.format(iter)).expect_partial()
    else:
        net.load_weights('../data/L63/1M/L63').expect_partial()
    return net



s = 1.0
# define drift
def mu(X):
    x, y, z = tf.split(X, [1, 1, 1], axis=-1)
    p = alpha * (y - x)
    q = x * (rho - s * z) - y 
    r = s * x * y - beta * z 
    return -tf.concat([p, q, r], axis=-1).numpy()

n_theta = get_net(1000000)

# define h0

def h0(X):
    l = len(X)
    m = int(1e5)
    M = int(np.ceil(l / m))
    data = []
    for i in range(M):
        if i < M-1:
            x_, y_, z_ = tf.split(X[i*m: (i+1)*m], [1, 1, 1], axis=-1)
        else:
            x_, y_, z_ = tf.split(X[i*m:], [1, 1, 1], axis=-1)
        log_p0 = (- (x_**2 + y_**2 + z_**2) / (2.*r**2)).numpy()
        log_pinf = n_theta(x_, y_, z_).numpy()
        data.append(np.exp(log_p0 - log_pinf) / (2. * np.pi * r**2) ** (1.5))  
    return np.concatenate(data, axis=0) 
    
def sol(X):
    l = len(X)
    m = int(1e5)
    M = int(np.ceil(l / m))
    data = []
    for i in range(M):
        if i < M-1:
            x_, y_, z_ = tf.split(X[i*m: (i+1)*m], [1, 1, 1], axis=-1)
        else:
            x_, y_, z_ = tf.split(X[i*m:], [1, 1, 1], axis=-1) 
        data.append(np.exp(n_theta(x_, y_, z_).numpy()))
        
    return np.concatenate(data, axis=0) 



grid = mc_prob.get_grid()
low = grid.mins
high = grid.maxs
n_subdivs = n_subdivisions
x = np.linspace(low[0], high[0], num=n_subdivs+1, endpoint=True).astype('float32')[1:]
y = np.linspace(low[1], high[1], num=n_subdivs+1, endpoint=True).astype('float32')[1:]
z = np.linspace(low[2], high[2], num=n_subdivs+1, endpoint=True).astype('float32')[1:]
x, y, z = np.meshgrid(x, y, z, indexing='ij')
x = x.reshape(-1, 1)
y = y.reshape(-1, 1)
z = z.reshape(-1, 1)
x_ = np.linspace(low[0], high[0], num=n_subdivs, endpoint=True).astype('float32')
y_ = np.linspace(low[1], high[1], num=n_subdivs, endpoint=True).astype('float32')
z_ = np.linspace(low[2], high[2], num=n_subdivs, endpoint=True).astype('float32')
iters = list(range(0, 500, 1)) + list(range(500, 10000, 100)) + [800000]*24
nets = {iter: get_net(iter) for iter in iters}

    
fig_all = plt.figure(figsize=(16, 16))
ax_1l = fig_all.add_subplot(321) 
ax_1m = fig_all.add_subplot(322)


ax_2l = fig_all.add_subplot(323) 
ax_2m = fig_all.add_subplot(324)

ax_3l = fig_all.add_subplot(325) 
ax_3m = fig_all.add_subplot(326)


div_1 = make_axes_locatable(ax_1l)
cax_1 = div_1.append_axes('right', '5%', '5%')
div_2 = make_axes_locatable(ax_2l)
cax_2 = div_2.append_axes('right', '5%', '5%')
div_3 = make_axes_locatable(ax_3l)
cax_3 = div_3.append_axes('right', '5%', '5%')

tick_size = 15
title_size = 20
cbar_tick_size = 15

im = ax_1m.pcolormesh(x_, y_, mc_prob.compute_p2(0, 1, save=False), cmap='inferno', shading='auto')
cbar = fig_all.colorbar(im, ax=ax_1m)
cbar.ax.tick_params(labelsize=cbar_tick_size)
#cbar.formatter.set_powerlimits((0, 0))
ax_1m.set_aspect("equal")

im = ax_2m.pcolormesh(y_, z_, mc_prob.compute_p2(1, 2, save=False), cmap='inferno', shading='auto')
cbar = fig_all.colorbar(im, ax=ax_2m)
cbar.ax.tick_params(labelsize=cbar_tick_size)
#cbar.formatter.set_powerlimits((0, 0))
ax_2m.set_aspect("equal")

im = ax_3m.pcolormesh(z_, x_, mc_prob.compute_p2(2, 0, save=False).T, cmap='inferno', shading='auto')
cbar = fig_all.colorbar(im, ax=ax_3m)
cbar.ax.tick_params(labelsize=cbar_tick_size)
#cbar.formatter.set_powerlimits((0, 0))
ax_3m.set_aspect("equal")

ax_1m.tick_params(axis='both', which='major', labelsize=tick_size)
ax_1m.tick_params(axis='both', which='minor', labelsize=tick_size)
ax_2m.tick_params(axis='both', which='major', labelsize=tick_size)
ax_2m.tick_params(axis='both', which='minor', labelsize=tick_size)
ax_3m.tick_params(axis='both', which='major', labelsize=tick_size)
ax_3m.tick_params(axis='both', which='minor', labelsize=tick_size)

ax_1m.set_title('Monte Carlo estimate', fontsize=title_size)
ax_1m.set_xlabel('x', fontsize=title_size)
ax_1m.set_ylabel('y', fontsize=title_size)
ax_2m.set_xlabel('y', fontsize=title_size)
ax_2m.set_ylabel('z', fontsize=title_size)
ax_3m.set_xlabel('z', fontsize=title_size)
ax_3m.set_ylabel('x', fontsize=title_size)



fk = sim.FKSim3_2(save_folder='.', n_subdivs=n_subdivs, n_int_subdivs=n_int_subdivs, mu=mu, sigma=sigma, sol=sol, grid=grid, h0=h0)
z_1l = fk.make_plot(n_steps, dt, n_repeats, 0, 1, 2)
# z_1l, z_2l, z_3l = fk.compile(n_repeats) 


ax_1l.clear()
cax_1.cla()
z_1l /= (z_1l.sum() * grid.h[0] * grid.h[1])
im = ax_1l.pcolormesh(x_, y_, z_1l, cmap='inferno')
ax_1l.set_title('Learned solution at time = {:.4f}'.format(t), fontsize=title_size)
ax_1l.tick_params(axis='both', which='major', labelsize=tick_size)
ax_1l.tick_params(axis='both', which='minor', labelsize=tick_size)
cbar = fig_all.colorbar(im, cax=cax_1, ax=ax_1l)
cbar.ax.tick_params(labelsize=cbar_tick_size)
#cbar.formatter.set_powerlimits((0, 0))
ax_1l.set_aspect("equal")
plt.savefig('L63-time-one-third.png')

z_2l = fk.make_plot(n_steps, dt, n_repeats, 1, 2, 0)

ax_2l.clear()
cax_2.cla() 
z_2l /= (z_2l.sum() *  grid.h[1] * grid.h[2])
im = ax_2l.pcolormesh(y_, z_, z_2l, cmap='inferno')
ax_2l.tick_params(axis='both', which='major', labelsize=tick_size)
ax_2l.tick_params(axis='both', which='minor', labelsize=tick_size)
cbar = fig_all.colorbar(im, cax=cax_2, ax=ax_2l)
cbar.ax.tick_params(labelsize=cbar_tick_size)
#cbar.formatter.set_powerlimits((0, 0))
ax_2l.set_aspect("equal")
plt.savefig('L63-time-two-third.png')

z_3l = fk.make_plot(n_steps, dt, n_repeats, 2, 0, 1)

ax_3l.clear()
cax_3.cla()
z_3l /= (z_3l.sum() *  grid.h[2] * grid.h[0])
im = ax_3l.pcolormesh(z_, x_, z_3l, cmap='inferno')
ax_3l.tick_params(axis='both', which='major', labelsize=tick_size)
ax_3l.tick_params(axis='both', which='minor', labelsize=tick_size)
cbar = fig_all.colorbar(im, cax=cax_3, ax=ax_3l)
cbar.ax.tick_params(labelsize=cbar_tick_size)
#cbar.formatter.set_powerlimits((0, 0))
ax_3l.set_aspect("equal")
fig_all.subplots_adjust(wspace=0.3, hspace=0.2)

ax_1l.set_ylabel(r'$p(x, y)$ at time {:.4f}'.format(t), fontsize=title_size)
ax_2l.set_ylabel(r'$p(y, z)$ at time {:.4f}'.format(t), fontsize=title_size)
ax_3l.set_ylabel(r'$p(z, x)$ at time {:.4f}'.format(t), fontsize=title_size)
plt.savefig('L63-time.png')

<IPython.core.display.Javascript object>

Time taken by compute_p2 is 2.9152135848999023 seconds
Time taken by compute_p2 is 2.949232339859009 seconds
Time taken by compute_p2 is 2.9029064178466797 seconds


In [3]:
z_1l

array([[2.18156901e-09, 2.88435967e-07, 3.87610891e-07, 5.30332917e-06,
        3.74237445e-07, 2.70603837e-07, 4.44300647e-08, 7.12238087e-13,
        1.01649444e-28, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00],
       [4.82299551e-08, 1.06566797e-06, 3.15577341e-06, 5.81584089e-06,
        2.45979493e-06, 6.19034528e-07, 1.66014393e-08, 4.67755522e-14,
        2.32871387e-21, 4.91169650e-22, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00,